In [1]:
import os
import pickle
import sys
from calendar import month
from datetime import date, datetime, timedelta
from importlib import reload
from pathlib import Path
from pprint import pprint
from time import time

import get_data
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytz
import quantstats as qs
import streamlit as st
from binance.client import Client
from datasets import assets, portfolios
from datasets.portfolios import PastPortfolio

In [2]:
api_key = os.environ.get("BINANCE_WATCH_API")
api_secret = os.environ.get("BINANCE_WATCH_SECRET")
client = Client(api_key, api_secret)

In [7]:
def _compute_current_balance(client):
    amount = {}
    value = {}
    for balance in client.get_account()["balances"]:
        current_amount = float(balance["free"]) + float(balance["locked"])
        if current_amount > 0:
            ticker = balance["asset"]
            if balance["asset"].startswith("LD"):
                ticker = balance["asset"][2:] 

            if ticker in ["BUSD", "USDT"]:
                avg_price = 1
            elif ticker == "BTC":
                avg_price = float(
                    client.get_avg_price(symbol=ticker + "USDT")[
                        "price"
                    ]
                )

            else:
                avg_price = float(
                    client.get_avg_price(symbol=ticker + "BTC")[
                        "price"
                    ]
                ) * float(client.get_avg_price(symbol="BTCUSDT")["price"])
            current_value = current_amount * avg_price
        else:
            current_value = 0
        amount[balance["asset"]] = current_amount
        value[balance["asset"]] = current_value
    return amount, value

_compute_current_balance(client)

({'BTC': 0.0,
  'LTC': 0.0,
  'ETH': 0.0,
  'NEO': 0.0,
  'BNB': 0.0,
  'QTUM': 0.0,
  'EOS': 0.0,
  'SNT': 0.0,
  'BNT': 0.0,
  'GAS': 0.0,
  'BCC': 0.0,
  'USDT': 0.0,
  'HSR': 0.0,
  'OAX': 0.0,
  'DNT': 0.0,
  'MCO': 0.0,
  'ICN': 0.0,
  'ZRX': 0.0,
  'OMG': 0.0,
  'WTC': 0.0,
  'YOYO': 0.0,
  'LRC': 0.0,
  'TRX': 0.0,
  'SNGLS': 0.0,
  'STRAT': 0.0,
  'BQX': 0.0,
  'FUN': 0.0,
  'KNC': 0.0,
  'CDT': 0.0,
  'XVG': 0.0,
  'IOTA': 0.0,
  'SNM': 0.0,
  'LINK': 0.0,
  'CVC': 0.0,
  'TNT': 0.0,
  'REP': 0.0,
  'MDA': 0.0,
  'MTL': 0.0,
  'SALT': 0.0,
  'NULS': 0.0,
  'SUB': 0.0,
  'STX': 0.0,
  'MTH': 0.0,
  'ADX': 0.0,
  'ETC': 0.0,
  'ENG': 0.0,
  'ZEC': 0.0,
  'AST': 0.0,
  'GNT': 0.0,
  'DGD': 0.0,
  'BAT': 0.0,
  'DASH': 0.0,
  'POWR': 0.0,
  'BTG': 0.0,
  'REQ': 0.0,
  'XMR': 0.0,
  'EVX': 0.0,
  'VIB': 0.0,
  'ENJ': 0.0,
  'VEN': 0.0,
  'ARK': 0.0,
  'XRP': 0.0,
  'MOD': 0.0,
  'STORJ': 0.0,
  'KMD': 0.0,
  'RCN': 0.0,
  'EDO': 0.0,
  'DATA': 0.0,
  'DLT': 0.0,
  'MANA': 0.0,
  '

In [4]:
client.get_account()["balances"]

[{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'GAS', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BCC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'USDT', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'HSR', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'OAX', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'DNT', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'MCO', 'free